In [ ]:
!git clone https://github.com/Francesco9932/rag_risiko

In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

!pip install accelerate
!pip install -q pypdf
!pip install -q spacy
!python3 -m spacy download it_core_news_sm

In [1]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./rag_risiko/Regolamento-Risiko.pdf")
pages = loader.load()

In [3]:
pages[1].page_content

'Il gioco strategico\nper la conquista\ndel mondo\nDa 3 a 6 giocatori\nEtà: da 10 anni in su\n2\n'

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter()
docs = splitter.split_documents(pages)

In [5]:
docs[2]

Document(page_content='3SCOPO DEL GIOCO\nRaggiungere per primi il proprio obiettivo segreto.\nCOMPONENTI\nUn piano di gioco, rappresentante un planisfero suddiviso \nin 42 territori, appartenenti ai 6 continenti. Su ogni \nterritorio è indicato un Punteggio Vittoria. Tale valore \nserve esclusivamente se decidete di utilizzare le nuove \nregole per il gioco a tempo ridotto.\n6 armate di diverso colore composte da piccoli carrarmati \n(del valore di un’armata ciascuno) e bandierine (del valore \ndi 10 armate) 3 dadi di colore rosso e 3 dadi di colore blu.\nUn mazzo di carte “Territori” così suddiviso:\n•\t42\tcarte\traffiguranti\t ciascuna\t uno\tdei\tterritori \n presenti sulla plancia di gioco e il simbolo di una \n delle tre armi dell’esercito (fante, cavallo, cannone).\n•\t2\tcarte\t“Jolly”\tdistinguibili\t dalle\taltre\tper\tla\tpresenza \n dei simboli di tutte le armi e l’assenza della mappa.\nUn mazzo di carte “Obiettivi”, indicanti gli obiettivi segreti \nche i giocatori dovrann

In [6]:
# Define the path to the pre-trained model you want to use
modelPath = "nickprock/sentence-bert-base-italian-uncased"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
# use cpu or gpu
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_kwargs = {'device': device}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
db = FAISS.from_documents(docs, embeddings)

In [19]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [29]:
# template="""
#
#
#                             \n\n\nPlayer Question:\n{input}"""

template = """
[INST] <>
You are an expert board game player and teacher. You have memorized all the rules to every board game ever made.
Use the rules in your memory along with any other context you have to answer the players questions below.
<>

{context}

{question} [/INST]
"""

In [30]:
prompt = PromptTemplate(template=template, input_variables=["context", "input"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [37]:
input = "Quanti sono i territori? Dammi una risposta coincisa con solo il numero di territori."
result_ = qa_chain(
    input
)
result = result_["result"].strip()


print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Quanti sono i territori? Dammi una risposta coincisa con solo il numero di territori.
There are 42 territories on the game board.


In [40]:
input = "Riassumi lo scopo del gioco, specificandone i punti chiave e numero massimo di giocatori. Devi essere coinciso"
result_ = qa_chain(
    input
)
result = result_["result"].strip()


print(input)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Riassumi lo scopo del gioco, specificandone i punti chiave e numero massimo di giocatori. Devi essere coinciso
The objective of the game is for each player to reach their own secret goal first. The game components include a game board representing a world map divided into 42 territories belonging to six continents, each territory having a victory point value that serves only if the new rules for reduced time play are used. There are six armies of different colors, each consisting of small infantry units (worth one army each) and banners (worth 10 armies). There are three red dice and three blue dice. There is also a deck of "Territory" cards, which depict each territory on the game board and the symbol of one of the three weapons of war. Additionally, there are two "Jolly" cards, which are distinguishable from other cards by the presence of symbols of all three weapons and the absence of a map. Finally, there is a deck of "Objective" cards, which indicate the secret goals that the play

In [ ]:
# # Questa parte da rivedere:

# # Specify the model name you want to use
# model_name = "mrm8488/bert-italian-finedtuned-squadv1-it-alfa"

# # Load the tokenizer associated with the specified model
# tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# # Define a question-answering pipeline using the model and tokenizer
# question_answerer = pipeline(
#     "question-answering",
#     model=model_name,
#     tokenizer=tokenizer,
#     return_tensors='pt'
# )

# # Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# # with additional model-specific arguments (temperature and max_length)
# llm = HuggingFacePipeline(
#     pipeline=question_answerer
# )

Some weights of the model checkpoint at mrm8488/bert-italian-finedtuned-squadv1-it-alfa were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# # Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
# retriever = db.as_retriever()

# # Create a question-answering instance (qa) using the RetrievalQA class.
# # It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [ ]:
# question = "quanti sono i territori?"
# result = qa.run({"query": question})
# print(result["result"])